# Phase 1: Data Collection & Data Preparation

## Ziel der Phase
In dieser Phase sammeln wir relevante Daten aus verschiedenen Quellen, bereinigen sie und bringen sie in eine konsistente Struktur, sodass daraus pro Fahrer und Saison Features berechnet werden können.  
Das Ziel ist ein reproduzierbarer Datenprozess, der am Ende modellfähige Tabellen liefert.

---

## Datenquellen und Abdeckung

### Serien
- Formel 3 (F3)
- Formel 2 (F2)
- Formel 1 (F1) als Referenz und für das spätere Labeling

### Grundidee
Wir trennen bewusst:
- Rohdaten aus externen Quellen
- Zwischenergebnisse nach Bereinigung
- finale Feature Tabellen pro Serie

Diese Trennung hilft bei Nachvollziehbarkeit, Debugging und Reproduzierbarkeit.

---

## Herkunft der Daten und Auswahl der Quellen

### Überblick
Die Daten für dieses Projekt stammen aus unterschiedlichen Quellen und wurden je nach Serie unterschiedlich erhoben.  
Die Auswahl der finalen Datenquellen erfolgte nicht nur aus praktischen Gründen, sondern auch auf Basis einer kritischen Qualitätsprüfung der verfügbaren Datensätze.

---

### Formel 1 (F1)

**Quelle:** Kaggle  
Die F1-Daten wurden aus einem öffentlich verfügbaren Kaggle-Datensatz bezogen.

**Begründung für die Nutzung:**
- Historisch vollständige Abdeckung vieler Saisons
- Klare Struktur mit Fahrer- und Saisoninformationen
- Geeignet zur Identifikation späterer F1-Einstiege

**Rolle im Projekt:**
Die F1-Daten werden nicht für das Modelltraining verwendet, sondern ausschliesslich:
- zur Bestimmung, ob ein Fahrer später in der Formel 1 gestartet ist
- zur Ermittlung des ersten F1-Jahres eines Fahrers (Labeling)

---

### Formel 2 (F2)

**Initiale Quelle:** Kaggle  

Zu Beginn wurden auch für die Formel 2 Daten aus Kaggle verwendet.  
Nach der ersten Datenaufbereitung zeigte sich jedoch ein wesentliches Problem:

- Der Datensatz enthielt Einträge für zukünftige Saisons
- Diese Einträge sind aus Prognosesicht nicht realistisch
- Die zeitliche Konsistenz der Daten war nicht gewährleistet

Da dies zu Datenleckage und unrealistischen Modellergebnissen führen könnte, wurde entschieden, diese Quelle nicht weiter zu verwenden.

**Finale Quelle:** https://www.fiaformula2.com/

**Vorgehen:**
- Offizielle Rennresultate wurden von der FIA Formula 2 Website extrahiert
- Es wurden ausschliesslich tatsächlich gefahrene Saisons berücksichtigt
- Die Daten wurden anschliessend bereinigt, vereinheitlicht und aggregiert

Diese Entscheidung stellt sicher, dass alle F2-Daten zeitlich korrekt und realistisch sind.

---

### Formel 3 (F3)

**Quelle:** https://www.fiaformula3.com/

Die Formel 3 Daten wurden direkt von der offiziellen FIA Formula 3 Website bezogen.

**Vorgehen:**
- Extraktion von Renndaten anhand eindeutiger Race IDs
- Zuordnung von Fahrern, Sessions und Resultaten
- Vereinheitlichung der Datenstruktur über mehrere Saisons hinweg
- Aggregation der Ergebnisse pro Fahrer und Saison

Durch die Verwendung der offiziellen Website konnte eine hohe Datenqualität und Konsistenz sichergestellt werden.

---

## Begründung der finalen Quellenwahl

Die finale Auswahl der Datenquellen basiert auf folgenden Kriterien:

- Zeitliche Korrektheit (keine zukünftigen Informationen)
- Reale, tatsächlich gefahrene Saisons
- Konsistente Struktur über mehrere Jahre
- Minimierung von Datenleckage

Insbesondere der Wechsel der F2-Datenquelle zeigt,
dass Datenqualität und methodische Korrektheit
höher gewichtet wurden als Bequemlichkeit oder Vollständigkeit.

---

## Projektstruktur und Ordnerlogik

### Datenordner
Unter `data/` verwenden wir pro Serie eine klare Struktur:

- `raw/`  
  Unveränderte Quelldaten (Download oder Extraktion)
- `interim/`  
  Bereinigte, normalisierte Zwischenstände
- `processed/`  
  Finale Outputs dieser Phase (Features pro Fahrer und Saison)

Zusätzlich existiert:
- `data/all_series/processed/`  
  Zusammengeführte Tabellen über alle Serien
- `data/model_input/`  
  Modellinput Dateien, die nicht mehr verändert werden sollen

---

## Datenpipeline pro Serie

### Ziel pro Serie
Aus Renn oder Sessiondaten erstellen wir eine Fahrer Saison Tabelle mit aggregierten Leistungskennzahlen, zum Beispiel:
- Anzahl Rennen
- Punkte und Punkteschnitt
- durchschnittliche Platzierung, bestes und schlechtestes Resultat
- Siege und Podestplätze
- Top 10 Anteile
- Ausfallquote
- Pace Indikatoren (z. B. best lap)
- Stabilität (Standardabweichungen)

---

## F3 Datenerhebung und Feature Engineering

### Vorgehen
1. Rohdaten sammeln oder extrahieren
2. Felder vereinheitlichen (Namen, Teamnamen, Zeitformate)
3. Zeitwerte und Gaps in numerische Sekunden konvertieren
4. Feature Engineering pro Fahrer und Saison
5. Speichern als `processed` Output

### Output
- `data/f3/processed/f3_features.csv`
- optional zusätzliche erweiterte Feature Dateien aus Experimenten

---

## F2 Datenerhebung und Feature Engineering

### Vorgehen
Analog zu F3, jedoch angepasst an die Verfügbarkeit und Struktur der F2 Daten.

Wichtige Schritte
- Normalisierung von Zeit und Ergebnis Feldern
- Aggregation pro Fahrer und Saison

### Output
- `data/f2/processed/f2_features.csv`

---

## F1 Referenzdaten

### Zweck
F1 Daten werden nicht direkt zum Training genutzt, sondern dienen als Referenz für:
- Identifikation, ob ein Fahrer später in F1 gestartet ist
- Berechnung des ersten F1 Jahres pro Fahrer

### Output
- `data/f1/processed/f1_features_core.csv` oder vergleichbarer Core Output

---

## Datenqualität und Konsistenz Checks

### Standard Checks
- Shape, Spaltenliste, Beispielzeilen
- Duplikate nach Schlüsseln (series, year, driver_code)
- Missing Values pro Spalte
- Wertebereiche plausibilisieren (z. B. Raten zwischen 0 und 1)
- Sonderzeichen und Platzhalter wie `\N` entfernen oder korrekt behandeln

### Ziel
Früh Fehler finden, bevor Modellierung startet.

---

## Zusammenführung über alle Serien

### Motivation
Um später ein einheitliches Modellsetup zu ermöglichen, werden die Feature Tabellen zusammengeführt.

Schritte
- Schema Angleichen
- Konsistente Spaltennamen sicherstellen
- Zusammenführen in eine Master Tabelle

Output
- `data/all_series/processed/all_series_master_features_core.csv`

---

## Modellinput Vorbereitung als Abschluss dieser Phase

### Motivation
Ab dem Modellinput sollen Daten nicht mehr laufend angepasst werden.  
Stattdessen wird ein stabiler Input Ordner verwendet, damit Ergebnisse reproduzierbar bleiben.

Beispiele
- F2 und F3 Features mit F1 Einstieg Label
  `data/model_input/f2_f3_features_with_f1_label.csv`

---

## Output der Phase

Artefakte dieser Phase
- Pro Serie: Feature Tabellen in `data/<serie>/processed/`
- Master Tabelle über alle Serien in `data/all_series/processed/`
- Stabile Modellinput Dateien in `data/model_input/`

Diese Phase stellt sicher, dass die Modellierung in Phase 2 auf einer sauberen, nachvollziehbaren Datenbasis aufsetzt.
